In [105]:
import ipywidgets as widgets
from traitlets import Unicode, validate
from IPython.display import display
from traittypes import Array
import io
import numpy as np


def array_to_json(ar, obj=None):
    iobyte = io.BytesIO()
    np.save(iobyte,ar)
    return iobyte.getvalue()

def from_json_to_array(value, obj=None):
	return np.array(value) if value else None


array_serialization = dict(to_json=array_to_json, from_json=from_json_to_array)


class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    
    _model_name = Unicode('HelloModel').tag(sync=True)
    _model_module = Unicode('hello').tag(sync=True)
    x = Array().tag(sync=True,**array_serialization)

In [187]:
%%javascript
require.undef('hello');
function asciiDecode(buf) {
        return String.fromCharCode.apply(null, new Uint8Array(buf));
}

function readUint16LE(buffer) {
        var view = new DataView(buffer);
        var val = view.getUint8(0);
        val |= view.getUint8(1) << 8;
        return val;
}

function fromArrayBuffer(buf,manager) {
      // Check the magic number
      //var buf = new ArrayBuffer(buf)
      //console.log(buf)
        
      console.log("l",buf.slice(1,6) )

      var magic = asciiDecode(buf.slice(0,6));
      console.log("La")
      //return "test"
      console.log("l",magic.slice(1,6) )
      if (magic.slice(1,6) != 'NUMPY') {
          throw new Error('unknown file type');
      }

      var version = new Uint8Array(buf.slice(6,8)),
          headerLength = readUint16LE(buf.slice(8,10)),
          headerStr = asciiDecode(buf.slice(10, 10+headerLength));
          var offsetBytes = 10 + headerLength;
          //rest = buf.slice(10+headerLength);  XXX -- This makes a copy!!! https://www.khronos.org/registry/typedarray/specs/latest/#5

      console.log(headerStr.toLowerCase().replace('(','[').replace(',),',']').replace('),',']').replace(/'/g,"\""))
      var info =  JSON.parse(headerStr.toLowerCase().replace('(','[').replace(',),',']').replace('),',']').replace(/'/g,"\""));
      
      // Intepret the bytes according to the specified dtype
      console.log(info)
      var data;
      if (info.descr === "|u1") {
          data = new Uint8Array(buf, offsetBytes);
      } else if (info.descr === "|i1") {
          data = new Int8Array(buf, offsetBytes);
      } else if (info.descr === "<u2") {
          data = new Uint16Array(buf, offsetBytes);
      } else if (info.descr === "<i2") {
          data = new Int16Array(buf, offsetBytes);
      } else if (info.descr === "<u4") {
          data = new Uint32Array(buf, offsetBytes);
      } else if (info.descr === "<i4") {
          data = new Int32Array(buf, offsetBytes);
      } else if (info.descr === "<f4") {
          data = new Float32Array(buf, offsetBytes);
      } else if (info.descr === "<f8") {
          data = new Float64Array(buf, offsetBytes);
      } else {
          throw new Error('unknown numeric dtype')
      }
    
      
      var shape = info.shape;
      if (shape.length == 2){
          var ndata = new Array(shape[0])
          
          for(var i = 0; i< shape[0]; i++){
              ndata[i] = data.slice(i*shape[1],(i+1)*shape[1])
          }
      }
      else{
          var ndata = data
      }
      return ndata ;

}

define('hello', ["jupyter-js-widgets"], function(widgets) {

    var deserialize_numpy_array = function (data){
        var ret = fromArrayBuffer(data.buffer);
        return ret
    };
    // Define the HelloView
    var HelloView = widgets.DOMWidgetView.extend({
        render : function(){
            console.log(this.model.get("x"))
            var data = this.model.get("x")
            console.log(data,typeof data[0][0] )
            if (typeof data[0][0] != "undefined")
                this.el.textContent = "2D "+data[1][0]
            else
                this.el.textContent = "1D "+data[0]
        }

    });
    
            
    var HelloModel =  widgets.WidgetModel.extend({


    }, {
    serializers: _.extend({
        x: { deserialize: deserialize_numpy_array },

    }, widgets.WidgetModel.serializers)
});                       


    return {
        HelloView: HelloView,
        HelloModel: HelloModel
    }
});

<IPython.core.display.Javascript object>

In [188]:
V1 = HelloWidget(x=np.array([5.]))
display(V1)
V2 = HelloWidget(x=np.array(([6.,4],[7.,3])))
display(V2)

In [189]:
%timeit -n 10  HelloWidget(x=np.zeros((1000, 1000)))

10 loops, best of 3: 23.7 ms per loop


In [190]:
%timeit -n 10  HelloWidget(x=np.zeros(1000000))

10 loops, best of 3: 24.5 ms per loop
